In [1]:
import json
from fastapi import FastAPI
import gensim

import scipy

import sys
sys.path.append('./models')

from fastapi.middleware.cors import CORSMiddleware

import pickle 
import os
import glob

In [2]:
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

In [5]:
app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [6]:
user_input = 'London Rishi'

In [7]:
def reated_documents(user_input):
    words = user_input.split()
    rel_docs = set()
    for word in words:
        myquery = {"word": word}
        results  = mycollection.find(myquery)
        for result in results:
            docs = result['doc_ids']
            rel_docs.update(docs)
            
    return list(rel_docs)

In [8]:
rel_docs = reated_documents(user_input)

In [9]:
rel_docs = list(rel_docs)

In [10]:
len(rel_docs)

325

In [11]:
documents  = []
urls = []
for idx in rel_docs:
    query = f"SELECT * FROM sites where id = {idx};"
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        doc_id = row[0]
        url = row[1]
        urls.append(url)
        text = row[2]
        documents.append(text)

In [12]:
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)


In [14]:
tokenized_query = user_input.split()
doc_scores_raw = bm25.get_scores(tokenized_query)
doc_scores = doc_scores_raw.argsort()[::-1]

In [36]:
doc_scores[:10]

array([226,  80, 269, 134, 125, 293, 187,  27,  18,  73], dtype=int64)

In [23]:
doc_id = rel_docs[226]

In [26]:
out_data = {}
for idx,score_id in enumerate(doc_scores,start=1):
    doc_id = rel_docs[score_id]
    url = urls[score_id]
    text = documents[score_id][:200]
    out_data.update({idx:{'rank':idx,'doc_id':doc_id,'url':url,'text':text }})

In [27]:
len(out_data)

325

In [28]:
json_data = json.dumps(out_data)

In [29]:
json_data

'{"1": {"rank": 1, "doc_id": 1610, "url": "https://www.bbc.com/news/uk-england-london-64803166", "text": "Tube strike: RMT announces London Underground walk out More London Underground staff are to strike on 15 March, Budget Day, a transport workers\' union has announced. The RMT has announced the walkout,"}, "2": {"rank": 2, "doc_id": 906, "url": "https://www.bbc.com/news/uk-northern-ireland-64795902", "text": "Brexit: What is the Stormont brake? This video can not be played Watch: How would the Stormont brake work? First came the backstop, then the protocol, now we can expect to hear a lot more about the \\"S"}, "3": {"rank": 3, "doc_id": 1747, "url": "https://www.bbc.com/news/uk-england-london-64811855", "text": "London: Three in 10 miss out on first-choice secondary school Three in 10 children in London missed out on a place at their top choice of secondary school. Lambeth, in south London, had the lowest pro"}, "4": {"rank": 4, "doc_id": 1349, "url": "https://www.bbc.com/news/uk-e

In [62]:
doc_id = rel_docs[2]

In [24]:
doc_id

1610

In [25]:
query = f"SELECT * FROM sites where id = {doc_id};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(1610, 'https://www.bbc.com/news/uk-england-london-64803166', 'Tube strike: RMT announces London Underground walk out More London Underground staff are to strike on 15 March, Budget Day, a transport workers\' union has announced. The RMT has announced the walkout, which is in a dispute over pensions, job losses and contractual agreements. Members will walk out on the same day that Aslef members are striking on the Tube network. A spokesperson for Transport for London (TfL) said: "We have not proposed changes to anyone\'s pensions." But the RMT\'s general secretary Mick Lynch said: "Our members will never accept job losses, attacks on their pensions or changes to working conditions in order to pay for a funding cut which is the government\'s political decision. "Tube workers provide an essential service to the capital, making sure the city can keep moving and work long hours in demanding roles. "In return they deserve decent pensions, job security and good working conditions and the RMT

## TESTING WORD2VEC

As mentioned in the DESM paper, we compute the centroid of documents and compute the cosine similarity of the query words with document centroid.

In [3]:
from models.desm import DESM_Scoring

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [5]:
user_input = 'Rishi London'

In [6]:
DESM_Scoring(user_input)[:10]

[(357, 0.5438157918094071),
 (235, 0.540018284813073),
 (1026, 0.5383321500358363),
 (857, 0.5227484157467395),
 (1286, 0.5210322223092848),
 (906, 0.5115197459081515),
 (1346, 0.5109159466735993),
 (257, 0.5091735163939375),
 (1535, 0.5069178872246354),
 (1877, 0.5000919710250191)]

In [13]:
DESM_Scoring(user_input,'out')[:10]

[(39, -0.17146164923906326),
 (855, -0.24527691792150774),
 (357, -0.2735211163412179),
 (235, -0.2741706123294878),
 (1026, -0.2753330854336459),
 (1346, -0.2876226656380233),
 (1286, -0.2963645362353523),
 (857, -0.2981949622265271),
 (257, -0.29885518581907633),
 (1748, -0.30637560548656806)]

In [11]:
query = f"SELECT * FROM sites where id = {1877};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(1877, 'https://www.bbc.com/news/uk-northern-ireland-64718730', 'Good Friday Agreement: Who wants to tell the \'truth\' about NI\'s past? It is almost 25 years since the signing of the Good Friday Agreement, the peace deal that brought an end to the Troubles - in the coming weeks, BBC News NI across our news website, TV and radio, will be looking back at the historic deal, speaking to the key players and reflecting on how the agreement continues to shape Northern Ireland. Take the legacy of the Troubles - an issue that dominates hearts and minds in 2023. A controversial UK government bill going through Westminster is offering conditional amnesties to perpetrators who co-operate with a new information recovery body - a kind of truth recovery process, not unlike the Truth and Reconciliation Commission set up in post-Apartheid South Africa. But, as former BBC security correspondent Brian Rowan finds out, there are big questions over who would be willing to tell the truth - and what the tr